# Thu thập dữ liệu:

- **Môi trường:** Google colab, Drive hoặc Local
- **Thư viện:** pandas, BeautifulSoup, requests, os, datetime
- **Các bước thu thập dữ liệu:**
  - **Bước 1:** Dùng requests để request trang web worldometers/coronavirus (https://www.worldometers.info/coronavirus/) để get url có thể sử dụng được cho BeautifulSoup html parser
  - **Bước 2:** Dùng soup để find 3 table cho 3 ngày theo ID ‘main_table_countries_{day} với day là từng item trong daylist gồm [`today`, `yesterday`,  `yesterday2`]
  - **Bước 3:** Lấy tên của các column bằng tab `th`, sau đó clean lại để thuận tiện hơn trong việc phân tích
  - **Bước 4:** Tạo thư mục chứa data theo nguyên tắc data_{today}
  - **Bước 5:** Read từng row bằng (`tr`) và từng item trong row (`td`) rồi append vào 1 list (list sau khi được append đến row cuối cùng sẽ được chuyển thành pd.DataFrame, khi này DF tương ứng 1 table cho 1 ngày trên worldometer) sau đó write DataFrame đó vào file csv (hoặc xlsx) trong folder data vừa tạo ở phía trên theo tên covid19_{date} với date là ngày mà dữ liệu được cập nhật trên worldometer. Tiếp tục cho 2 ngày còn lại. \
    -> Sau khi hoàn tất các bước này, sẽ tạo ra 1 folder tên data_{date} gồm 3 file là 3 table của 3 ngày được hiển thị trên trang web.

# Colab (uncomment khi chạy trên Google Colab)
---

In [1]:
# %%capture
# !pip install selenium
# # !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd gdrive/MyDrive/data-visualization/

In [4]:
# import requests
# import json

# url = 'http://ipinfo.io/json'
# response = requests.get(url)
# data = json.loads(response.text)

# print(data['city'], data['country'], data['timezone'])

---

# Local

In [5]:
import pandas as pd
import numpy as np

from datetime import date, timedelta
from bs4 import BeautifulSoup
import requests

import os

In [6]:
url=requests.get('https://www.worldometers.info/coronavirus/').text
soup=BeautifulSoup(url, 'html.parser')

In [7]:
daylist = [
    'today', 'yesterday', 'yesterday2'
]

my_tables = [soup.find('table', attrs={'id':f'main_table_countries_{day}'}) for day in daylist]

HTML Table Elements
- `th` table header
- `tr` table row
- `td` table data

In [8]:
rules = {
    ' ': '_',
    '/': '_',
    '\n': '',
    '\xa0': '_',
    ',': '_',
    '#': 'index'
}

def clean_col(text):
    for k, v in rules.items():
        text = text.replace(k, v)
    return text.lower()

col_extract=my_tables[0].findAll('th') # th = table header
columns_ = {
    col_id: clean_col(col_extract[col_id].text) for col_id in range (len(col_extract))
}
columns_

{0: 'index',
 1: 'country_other',
 2: 'totalcases',
 3: 'newcases',
 4: 'totaldeaths',
 5: 'newdeaths',
 6: 'totalrecovered',
 7: 'newrecovered',
 8: 'activecases',
 9: 'serious_critical',
 10: 'tot_cases_1m_pop',
 11: 'deaths_1m_pop',
 12: 'totaltests',
 13: 'tests_1m_pop',
 14: 'population',
 15: 'continent',
 16: '1_caseevery_x_ppl',
 17: '1_deathevery_x_ppl',
 18: '1_testevery_x_ppl',
 19: 'new_cases_1m_pop',
 20: 'new_deaths_1m_pop',
 21: 'active_cases_1m_pop'}

In [9]:
data_dir=f'data_{date.today()}'

# create data directory
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [10]:
delta=0
for table in my_tables:
    current_table=[]
    for row in table.findAll('tr'): # tr = table row
        row_data=[]
        for cell in row.findAll('td'): # td = table data
            row_data.append(cell.text)
            
        if(len(row_data) > 0):
            data = {
                columns_[i]: row_data[i] for i in range(0, len(row_data))
            }
            current_table.append(data)
    df=pd.DataFrame(current_table)

    date_=(date.today() - timedelta(days=delta)).strftime("%d_%m_%Y")
    filename=f'{data_dir}/covid19_{date_}.csv'
    
    # .csv
    df.to_csv(filename, index=False)
    
    # # .xlsx
    # df.to_excel(filename, index=False)
    
    print(f'{delta}. {filename.replace("/", "|")}')
    delta+=1

0. data_2023-03-15|covid19_15_03_2023.csv
1. data_2023-03-15|covid19_14_03_2023.csv
2. data_2023-03-15|covid19_13_03_2023.csv
